In [55]:
import time
start_time = time.time()
import os
import re
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from slugify import slugify

# Determining the path where the files are located and creating a new folder
pathn = '/content/drive/MyDrive/BIMJSON/TGA2'

##############################################################################

# Sets the type for geometry grouping
# Category, Сomment, Type or any other property of any element

search_parameter = 'Type'

##############################################################################

# Main function for grouping data and saving a file
def crtable(filename):
    filenamep = pathn + '/' + filename
    df = pd.read_csv(filenamep, low_memory=False)
    filedae = pathn + '/' + filename[:-8]+'dae'
    ### grouping by element types for different formats
    #if 'ifc' in filenamep:
    #  search_parameter = 'Type'
    #else:
    #  search_parameter = 'Type Name'

    # Determination of the number of elements in groups
    df2= df.groupby([search_parameter])['Volume'].agg(['count'])
    dfallpar = pd.DataFrame()  
    df['Unnamed: 0'] = df['Unnamed: 0'].astype(str)
    comma = lambda x: ', '.join(x.unique())
    df3 = df.groupby([search_parameter]) .agg({'Unnamed: 0': comma})
    
    # Collecting data into one dataframe
    dfallpar = pd.concat([df2, df3], axis=1)
    dfallpar.rename(columns=({ 'Unnamed: 0': 'Id´s', 'count': 'Amount'}), inplace=True,)
    
    dfallpar['type'] = dfallpar.index
    #dfallpar.insert(0, 'Group Key', dfallpar['type'].apply(slugify))
    
    # Start sorting geometry from DAE file
    # Formation of a data tree from the DAE format
    daegrpath = pathn + '/' + 'DAEgroups_' + filename[:-9] 
    try:
      os.mkdir(daegrpath)
    except:
      pass

    # If the ID of an element from the group_ids_str list that was found earlier matches,
    # all elements with this ID are found in the DAE file, and all other elements are deleted
    filedaearr = []

    for index, row in dfallpar.iterrows():
      fileObject = open(filedae, "r")
      treeq = ET.parse(fileObject)
      root = treeq.getroot()
      ET.register_namespace("", "http://www.collada.org/2005/11/COLLADASchema")
      geom_list = []
      group_ids_str = []
      group_ids_str = re.findall(r'\d+', row['Id´s'])
      for node in root.findall('.//{http://www.collada.org/2005/11/COLLADASchema}node'):
          tree = treeq
          if node.attrib['id'] in group_ids_str:
            try:
              url = list(node)[0].get('url')
              geom_list.append(url[1:])
            except:
              pass
          else:
              try:
                  nd = node.find(
                      '{http://www.collada.org/2005/11/COLLADASchema}instance_geometry')
                  node.remove(nd)
              except:
                  0
      for geomet in root.findall('.//{http://www.collada.org/2005/11/COLLADASchema}geometry'):
            if geomet.attrib['id'] in geom_list:
                0
            else:
                try:
                  md = geomet.find(
                      '{http://www.collada.org/2005/11/COLLADASchema}mesh')
                  geomet.remove(md)
                except:
                  pass
      # Formation of a new name for the DAE file with grouped elements
      # words_pattern = '[a-zA-Z10-9]+'
      invalid = '<>:"/\|?* '
      for char in invalid:
        index = index.replace(char, '')
      regw = index + '.dae'
      filedaena = daegrpath + '/' + regw
      with open(filedaena, 'w') as f:
          tree.write(f, encoding='unicode')

# Function execution cycle for all CSV files in the folder
for filename in os.listdir(pathn):
  if filename.endswith("csv"): 
      crtable(filename)
      
print("--- %s seconds ---" % (time.time() - start_time))
# Saving data to a ZIP file for downloading to a computer
#!zip -r /content/file.zip /content/rvt

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
